# Scraping Tools



## LLM Tools




### Article Extract



#### the code

extract llm article?


In [ ]:
const getClient = importer.import('selenium client')
//const llama = importer.import('llm code')
const selectDom = importer.import('selenium select')

// TODO: help me extract this article from html, only return the article in plain text and remove the html:


async function extractArticle(driver, startPage, retry) {
  //let llmCode = await llama

  if (!driver)
    driver = await getClient()

  if (!startPage) {
    return
  }

  try {
    console.log('loading page ', startPage)

    await driver.get(startPage)

    await new Promise(resolve => setTimeout(resolve, 1500))

    let bodyElements = await selectDom(driver, ['//body//*[text() and not(self::script|self::style|self::form) and not(ancestor::aside|ancestor::nav|ancestor::form|ancestor::header)]'])
    let bodyText = []

    for (let i = 0; i < bodyElements.length; i++) {
      try {
        let text = await bodyElements[i].getText()
        bodyText.push(text)
      } catch (e) {
        if (e.message.includes('stale element reference')) {
          continue
        } else {
          throw e
        }
      }
    }

    return bodyText
      .map(t => Array.isArray(t) ? t.join('\n').trim() : t.trim())
      .filter(t => t.length)
      .join('\n')
    /*
        let q1 = 'Extract the text content from this HTML page without forming any new HTML code:\n'
          + bodyElements.join('\n') + '\nReturn plain text extracted from the page and nothing else.'
    
        console.log('User: ' + q1)
        let a1 = await llmCode(q1)
        console.log('AI: ' + a1)
    
        let code = a1.matchAll(/```(markdown)*\n[\s\S]*?\n```/gi)
    
        // extract code blocks from response
        let codeBlocks = ''
        for(let match of code) {
          codeBlocks += match[0].replace(/^```(markdown)*\n|\n```$/gi, '') + '\n'
        }
        if(!codeBlocks) {
          console.log('Error, couldn\'t find code in:' + a1)
          return
        }
    
        return codeBlocks.join('\n')
    */

  } catch (up) {
    if (!up.message.includes('page crash')) {
      driver.quit()

      throw up
    } else if(!retry) {
      return await extractArticle(driver, startPage, true)
    } else {
      throw up
    }
  }
}


module.exports = extractArticle



test article extract?


In [ ]:


const extractArticle = importer.import('extract llm article')
const getClient = importer.import('selenium client')

async function testExtractor(startPage) {
  if(!startPage) {
    startPage = 'https://tsakraklides.com/2025/02/05/in-the-age-of-infinite-consumer-choice-the-only-choice-is-collapse/'
  }

  driver = await getClient()

  let result = await extractArticle(driver, startPage)

  driver.quit()

  return result
}


module.exports = testExtractor





### Summarizer



#### the code

summarize llm article?



In [ ]:

const llama = importer.import('create llm session')

// TODO: prompt llm, in one sentence summarize this article. in a whole paragraph summarize this article.

async function summerizeArticle(article, funny) {
  let {llmPrompt} = await llama

  if(!funny) {
    funny = ''
  }

  let q1 = 'Summerize this article in a single paragraph ' + funny + ':\n' + article.substr(0, 4096) + '\nOnly return the summary and nothing else, no explanations.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)


  let q2 = 'Summerize this article in a single sentence ' + funny + ':\n' + article.substr(0, 4096) + '\nOnly return the summary and nothing else, no explanations.'
  console.log('User: ' + q2)
  let a2 = await llmPrompt(q2)
  console.log('AI: ' + a2)

  return [a1, a2]
}


module.exports = summerizeArticle



test article summarizer?



In [ ]:
const extractArticle = importer.import('extract llm article')
const getClient = importer.import('selenium client')
const summerizeArticle = importer.import('summarize llm article')

async function testExtractor(startPage) {
  if (!startPage) {
    startPage = 'https://tsakraklides.com/2025/02/05/in-the-age-of-infinite-consumer-choice-the-only-choice-is-collapse/'
  }

  driver = await getClient()

  try {

    let result = await extractArticle(driver, startPage)

    driver.quit()

    let summary = await summerizeArticle(result)

    return summary
  } catch (e) {
    driver.quit()

    throw e
  }
}


module.exports = testExtractor



### Summarize All



#### the code

summarize all articles?



In [ ]:
const getClient = importer.import('selenium client')
const extractArticle = importer.import('extract llm article')
const summerizeArticle = importer.import('summarize llm article')
const {
  defaultCollector, persistSummaries
} = importer.import('default link collector')

// select link scraping tool
function selectScaper(selector, startPage) {
  if(!selector && startPage.includes('reddit.com')) {
    selector = importer.import('reddit month of links')
  } else if(!selector) {
    selector = defaultCollector
  } else if(typeof selector == 'string') {
    selector = defaultCollector.bind(null, startPage, selector)
  }

  return selector
}

// extract persist, extract persist
async function summerizeAll(links, selector, startPage, funny) {

  if(!startPage && !links) {
    console.error('No start page or links to summerize.')
    return
  }

  let selectorFunction = selectScaper(selector, startPage)

  let driver = await getClient()

  let summaries = persistSummaries(funny)

  try {
    if(!links && startPage) {
      links = await selectorFunction(driver, startPage)
    }

    console.log(links)

    for (let i = 0; i < links.length; i++) {
      if(typeof summaries[links[i].link] != 'undefined')
        continue // already loaded

      let article = await extractArticle(driver, links[i].link)

      let summary = await summerizeArticle(article, funny)
      
      summaries[links[i].link] = summary
      persistSummaries(funny, summaries)
    }

    driver.quit()

    return summaries
  } catch (e) {
    //driver.quit()
    throw e
  }
}

module.exports = summerizeAll



### Convert Summaries



#### the code

convert summaries?

TODO: rewrite this to be more useful



In [ ]:
const fs = require('fs')
const path = require('path')
const { safeurl } = importer.import('domain cache tools')
const { getNearestSunday } = importer.import('default link collector')
const summerizeArticle = importer.import('summarize all articles')
const llama = importer.import('create llm session')

const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'reasonings')

const CONVERSION_PROMPTS = [
  'summary',
  'pretend you\'re living in a star wars themed universe and reverse sentence structure like Yoda',
  'pretend you\'re a super intelligent alien species and write a metaphor for how they could be teasing us',
  'break the news to me as gently as possible, mommy me a little and tell me it\'s going to be okay',
  'Rewrite this news story as if it were a stand-up comedy routine, complete with witty punchlines and audience reactions',  
  'Make this breaking news sound like a script for a sitcom, with awkward misunderstandings, dramatic pauses, and a laugh track',  
  'Turn this serious news article into a rhyming Dr. Seuss-style poem, making it lighthearted but still informative',  
  'Retell this news story from the perspective of a sarcastic but lovable talking animal who’s just trying to make sense of the human world',  
  'Rewrite this news article as if it were an absurd conspiracy theory from a tin-foil-hat-wearing enthusiast',  
  'Make this news sound like an over-the-top tabloid headline, with wild exaggerations and celebrity name-drops',  
  'Rephrase this bad news as if you were a kindergarten teacher gently explaining it to a room full of toddlers',  
  'Write this difficult news as if it were a whimsical fairy tale where things work out in the end (or at least have a hopeful moral)',  
  'Turn this stressful news into a motivational speech, making it sound like an inspiring challenge rather than a disaster',  
  'Rewrite this news as if a wise old grandma was breaking it to you over a warm cup of tea and cookies',  
  'Make this bad news sound like a quirky horoscope, full of mystical optimism and cosmic reassurance',  
  'Transform this news into an uplifting bedtime story where even the roughest parts have a silver lining'
]

async function alternativeSummary(summary, funny) {
  let { llmPrompt } = await llama

  let q1 = 'Convert this article summary, ' + funny + ':\n' + summary[0] + '\nOnly return the summary and nothing else, no explanations.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)


  let q2 = 'Convert this article description, ' + funny + ':\n' + summary[1] + '\nOnly return the single sentence and nothing else, no explanations.'
  console.log('User: ' + q2)
  let a2 = await llmPrompt(q2)
  console.log('AI: ' + a2)

  return [a1, a2]
}

async function alternativeArticles(startPage, random) {

  // TODO: get base summary, if it doesn't exist generate it
  const CONVERSION_FILES = CONVERSION_PROMPTS.map(funny =>
    path.join(PROJECT_PATH, safeurl(getNearestSunday()) + '-' + safeurl(funny) + '.json'))


  // TODO: get a list of all other summary conversions
  const allSummaries = []
  for (let i = 0; i < CONVERSION_FILES.length; i++) {
    if (fs.existsSync(CONVERSION_FILES[i]))
      allSummaries[i] = JSON.parse(fs.readFileSync(CONVERSION_FILES[i]))
    else
      allSummaries[i] = {}
  }

  if (Object.values(allSummaries[0]).length == 0) {
    allSummaries[0] = await summerizeArticle(startPage)
  }

  // TODO: convert the base summaries to funny conversions and save
  let links = Object.keys(allSummaries[0])
  if(random) {
    random = Math.round(Math.random() * (CONVERSION_PROMPTS.length - 1)) + 1
  }
  for (let i = 0; i < links.length; i++) {
    for (let j = random ? random : 1; j < CONVERSION_PROMPTS.length; j++) {

      if(typeof allSummaries[j][links[i]] != 'undefined') {
        continue
      }

      // TODO: persist
      allSummaries[j][links[i]] = await alternativeSummary(allSummaries[0][links[i]], CONVERSION_PROMPTS[j])
      fs.writeFileSync(CONVERSION_FILES[j], JSON.stringify(allSummaries[j], null, 4))
      
      if(random) {
        break
      }
    }
  }

  return allSummaries
}


module.exports = {
  alternativeArticles,
  alternativeSummary,
  CONVERSION_PROMPTS,
}




## Scaper Tools



### Default Collector



#### the code

default link collector?


In [ ]:
const fs = require('fs')
const path = require('path')
const {safeurl} = importer.import('domain cache tools')

const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'reasonings')

function getNearestSunday(date = new Date()) {
  const day = date.getDay();
  const diff = date.getDate() - day + (day === 0 ? -7 : 0); // adjust when it is Sunday
  const result = new Date(date.setDate(diff));
  const customString = `${result.getMonth() + 1}/${result.getDate()}/${result.getFullYear()}`; 
  return customString
}

async function defaultCollector(driver, startPage, selector = '//a[@href]/@href') {
  const selectDom = importer.import('selenium select')
  const getClient = importer.import('selenium client')
  if(!driver)
    driver = getClient()

  try {
    await driver.get(startPage)
    await new Promise(resolve => setTimeout(resolve, 1000))
    let links = await selectDom(driver, selector)
    return links.map(l => ({link: l})) // to match reddit post lister
  } catch (e) {
    driver.quit()
    throw e
  }
}

// record previously generated summaries so script will eventually complete
function persistSummaries(funny, summaries) {
  if(!funny) {
    funny = 'summary'
  }
  let weeklySummary = path.join(PROJECT_PATH, safeurl(getNearestSunday()) + '-' + safeurl(funny) + '.json')
  if(!summaries && fs.existsSync(weeklySummary)) {
    return JSON.parse(fs.readFileSync(weeklySummary))
  } else if (!summaries) {
    return {}
  } else {
    fs.writeFileSync(weeklySummary, JSON.stringify(summaries, null, 4))
  }
}

module.exports = {
  defaultCollector,
  getNearestSunday,
  persistSummaries,
}
